# Working with text data

### Types of data represented as strings

Text is usually a string in your dataset, but not all string features are text.

For types of string data you might see:
* Categorical data
* Free strings that can be semantically mapped to categories
* Structured string data
* Text data

## Example Application: Sentiment Analysis of Movie Reviews

### Representing Text Data as a Bag of Words

Computing the bag-of-words representation fro a corpus of documents consist of 3 steps:
* Tokenization: split each document into the words that appear in it(called tokens)
* vocabulary building: collect a vocabulary of all words that appear in any of the documents, and number them
* Encoding: for each document, count how often each of the words in the vocabulary appear in this document

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
bards_words = ['The fool doth think he is wise', 'but the wise man knows himself to be a fool']
vect = CountVectorizer().fit(bards_words)

In [16]:
print('Vocabulary size: {}'.format(len(vect.vocabulary_)))
print('Vocabulary content: \n{}'.format(vect.vocabulary_))

Vocabulary size: 13
Vocabulary content: 
{'the': 9, 'fool': 3, 'doth': 2, 'think': 10, 'he': 4, 'is': 6, 'wise': 12, 'but': 1, 'man': 8, 'knows': 7, 'himself': 5, 'to': 11, 'be': 0}


In [17]:
bag_of_words = vect.transform(bards_words)
print('bag of words: \n{}'.format(bag_of_words.toarray()))

bag of words: 
[[0 0 1 1 1 0 1 0 0 1 1 0 1]
 [1 1 0 1 0 1 0 1 1 1 0 1 1]]


In [18]:
feature_names = vect.get_feature_names()
print('Number of features: {}'.format(len(feature_names)))
print('Feature names: {}'.format(feature_names))

Number of features: 13
Feature names: ['be', 'but', 'doth', 'fool', 'he', 'himself', 'is', 'knows', 'man', 'the', 'think', 'to', 'wise']


Note: CountVectorizer does not collect any statistics of the training part of the test part of the training data, so it can be used on all the training data befor cv, however, you still can not use fit it on the test data, and it is still recommanded to put it in pipeline and gridsearch the whole pipe.

By default, the regular expression that is used is '\b\w\w+\b', which matches all sequences of characters that consist of at least 2 letters or numbers and separated by word boundarys. It does not find single-letter words, and split up contractions like "doesn't", "bit.ly", but matches "h8ter" as a single word.

It then converts all words to lowercase.

This works quite well in practice, but often generates many uninformative features.

One way to cut back on these is to only use token that appears in at least 2(or 3, or 5) documents, this can be set with the min_df parameter

If the transform method is called on a document that contains words that were not contained in the training data, these words will be ignored. This is not really an issue for classification, as it's not possible to learn anything about words that are not in the training data.

Another way that we can get rid of uninformative words is by discarding words that are too frequent to be informative.

In [19]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [24]:
print('Number of stop words: {}'.format(len(ENGLISH_STOP_WORDS)))
print('Every 10th stop word:\n{}'.format(list(ENGLISH_STOP_WORDS)[::10]))

Number of stop words: 318
Every 10th stop word:
['a', 'etc', 'empty', 'enough', 'part', 'per', 'since', 'towards', 'whom', 'own', 'always', 'several', 'forty', 'but', 'anywhere', 'of', 're', 'mostly', 'none', 'yourself', 'ours', 'namely', 'keep', 'five', 'those', 'than', 'interest', 'although', 'if', 'themselves', 'because', 'without']


Except for specifying stop words, youcan also use max_df parameter to control the words you want to get rid of.

### Rescaling the data with tf-idf(term frequency=inverse document frequency)

Intuition: give high weight to any term that appears often in a particular document, but not in very many documents.
* TfidfTransformer: takes in the sparse matrix output produced by CountVectorizer and transforms it
* TfidfVectorizer: takes in the text data and does both the bag of words feature extraction and tf-idf transformation

tf-idf score: tfidf(w, d) = tf * log((N + 1) / (Nw + 1)) + 1
* N - number of documents in the training set
* Nw - number of ducuments in the training set that the word w appears in
* tf(term frequency) - number of times this word w appears in this document d
* d - this query document


Both class also apply l2 normalization after computing the tf-idf representation, in other words, they rescale the vector of each document to have euclidean norm 1.Rescaling in this way means that the length of a document(the number of words) does not change the vectorized representation.

Note: tf-idf makes use of the statistical properties of the training data, it must be used in a pipeline.

Note: tf-idf scaling is meant to find words that distinguish documents, but it is a purely unsupervised technique. So, the high-rating words found by it is not necessarily relate to the target.

For example, in movie reviews, some movie title may appear frequently in a subset of all the reviews, but seldomly appears in others, this has nothing to do with whether the review is positive or negative, but they will get high tf-idf scores. However, some words that are helpful, like "good", "great", may get relatively low ratings because they appear in too many reviews.

### Investigating Model Coefficients

We clearly cannot look at all of the coefficients at the same time. However, we can look at the largest coefficients and see ehich words these correspond to.

### Bag-of-Words with More Than One Word(n-Grams)

One of the main disadvantages of using a bag-of-words is that word order is completely discarded. Therefore, "It's bad, not good at all" and "It's good, not bad at all" will end up with same representation, even though the meanings are inverted.

Wecan capturing context when using bag-if-words representation by not only considering the count of single tokens, but also the count of pairs of tokens that appear next to each other.
- pairs of token -- bigrams
- triples of token -- trugrams

CountVectorizer and TfidfVectorizer got a ngram_range parameter for this

In [25]:
cv = CountVectorizer(ngram_range=(1, 1)).fit(bards_words)
print('Vocabulary size: {}'.format(len(cv.vocabulary_)))
print('Vocabulary:\n{}'.format(cv.get_feature_names()))

Vocabulary size: 13
Vocabulary:
['be', 'but', 'doth', 'fool', 'he', 'himself', 'is', 'knows', 'man', 'the', 'think', 'to', 'wise']


In [26]:
cv = CountVectorizer(ngram_range=(1, 2)).fit(bards_words)
print('Vocabulary size: {}'.format(len(cv.vocabulary_)))
print('Vocabulary:\n{}'.format(cv.get_feature_names()))

Vocabulary size: 27
Vocabulary:
['be', 'be fool', 'but', 'but the', 'doth', 'doth think', 'fool', 'fool doth', 'he', 'he is', 'himself', 'himself to', 'is', 'is wise', 'knows', 'knows himself', 'man', 'man knows', 'the', 'the fool', 'the wise', 'think', 'think he', 'to', 'to be', 'wise', 'wise man']


For most applications, the minimum number of tokens should be one, as single
words often capture a lot of meaning. Adding bigrams helps in most cases. Adding
longer sequences—up to 5-grams—might help too, but this will lead to an explosion
of the number of features and might lead to overfitting, as there will be many very
specific features.

### Topic Modeling and Document Clustering

- each sample to a single topic -- clustering
- each sample to multiple topics -- decomposition, each of the components we learned corresponds to one topic, and the coefficients of the components in the new repesentation of the sample tell us how strongly related the sample is to a particular topic.

## Summary

In test-processing applications, it is often possible to introspect models in a meaningful way, and this is true for both supervised and unsupervised learning tasks. You should take full advantage of this ability when using NLP-based methods in practice.